In [1]:
# Place to dowload book free
#https://annas-archive.org/
import spacy
import ebooklib
from ebooklib import epub  
from html.parser import HTMLParser #(usar beutiful soup)
from collections import defaultdict
from spacy.lang.es import Spanish
# En spacy, los saltos de linea los ignora, asi que supongo que para separar en parrafos o bien ingnorarlos, o bien procesarlos por separado,
# haciendo un split por \n



In [2]:
# paths and data acquisition 
book_path = ".\Asesinato en el Orient Express -- Agatha Christie -- Hércules Poirot 10, 1934 -- cba8798523cc82925c66887e83cdaa07 -- Anna’s Archive.epub"
book = epub.read_epub(book_path)


<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\pable\AppData\Local\Temp\ipykernel_7164\117803313.py:2: SyntaxWarning: invalid escape sequence '\A'
  book_path = ".\Asesinato en el Orient Express -- Agatha Christie -- Hércules Poirot 10, 1934 -- cba8798523cc82925c66887e83cdaa07 -- Anna’s Archive.epub"
c:\Users\pable\miniconda3\envs\main_env\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
c:\Users\pable\miniconda3\envs\main_env\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']})

In [3]:
title = book.get_metadata('DC', 'title')[0][0]
title

'Asesinato en el Orient Express'

In [4]:
# class to store the chapters
class Chapter():
    def __init__(self,chapter_name,chapter_text) -> None:
        self.chapter_name = chapter_name
        self.chapter_text = chapter_text


In [5]:
class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)

    def handle_endtag(self, tag):
        print("Encountered an end tag :", tag)

    def handle_data(self, data):
        print("Encountered some data  :", data)

parser = MyHTMLParser()
parser.feed('&#13;<p class="asangre salto35"><span class="letra">N</span>O llevaba sombrero. Entró con la cabeza echada hacia atrás, como en un desafío. La curva de su nariz recordaba una nave surcando valiente un mar embravecido. En aquel momento, Mary Debenham estaba hermosísima.</p>&#13;&#13;')

Encountered some data  : 
Encountered a start tag: p
Encountered a start tag: span
Encountered some data  : N
Encountered an end tag : span
Encountered some data  : O llevaba sombrero. Entró con la cabeza echada hacia atrás, como en un desafío. La curva de su nariz recordaba una nave surcando valiente un mar embravecido. En aquel momento, Mary Debenham estaba hermosísima.
Encountered an end tag : p
Encountered some data  : 


&#13;
  <h2 class="oculto" title="7. La identidad de Mary Debenham"/>&#13;
&#13;
  <div class="capitulo centrado">&#13;
    <p class="salto30">7</p>&#13;
&#13;
    <p class="salto05">LA IDENTIDAD DE MARY DEBENHAM</p>&#13;
  </div>&#13;
&#13;
  <p class="asangre salto35"><span class="letra">N</span>O llevaba sombrero. Entró con la cabeza echada hacia atrás, como en un desafío. La curva de su nariz recordaba una nave surcando valiente un mar embravecido. En aquel momento, Mary Debenham estaba hermosísima.</p>&#13;
&#13;
  <p>Su mirada se posó en Arbuthnot un instante…, sólo un instante.</p>&#13;
&#13;
  <p>—Deseaba preguntarle, señorita, por qué nos mintió usted esta mañana.</p>&#13;
&#13;
  <p>—¿Mentirle yo? No sé a lo que se refiere.</p>&#13;
&#13;

In [6]:
# function to get an orderer array of Chapters until a selected chapter
for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    if item.is_chapter():
        bodyContent = item.get_body_content().decode()
        print()
        f = HTMLFilter()
        f.feed(bodyContent)
        content += f.text

NameError: name 'HTMLFilter' is not defined

In [7]:
def search_adj_for_person(person_name,nlp_sentence):
    '''
    return all the adjectives in the sentence that are reffering to the person_name 
    '''
    adjectives = []
    for token in nlp_sentence:

        # If we find a character in the phrase or a NOUN, we get it
        if token.pos_  == "PROPN":
            character = token.text
        
        elif token.pos_  == "NOUN":
            character = None

        
        # If the token is an adjective, and its refering the character we want, we save them
        if character == person_name and token.pos_ == 'ADJ':
            adjectives.append(token.text)

    return adjectives

In [11]:
# Load the data
text = "Pablo es alto y guapo y tiene una silla bonita, mientras que Juan es bajo y feo y come muy mal"
nlp = spacy.load('es_core_news_md')

# Preprocess the text
doc = nlp(text)

# Initialize containers to store character information
character_info = defaultdict(lambda: {
    "description": [],
    "sentences": [],
    "mentions": 0,
    "relationships": defaultdict(int),
    "actions": []
})

# Iterate through the sentences to extract relevant information
for sent in doc.sents:
    sent_doc = nlp(sent.text)
    characters_in_sentence = []
    
    # We analyze the sentence token by token
    for token in sent_doc:
        # if there is an entity of type person we add all adjectives in the sentence as the description of the entity, until we find another person
        if token.ent_type_ == "PER":
            character_name = token.text
            
            # Add the sentence to the sentences list of that character
            character_info[character_name]["sentences"] += sent.text

            # Add the adjectives to the adjectives list of that character
            character_info[character_name]["description"] += search_adj_for_person(character_name,sent_doc)

            # Add the thigs he migth tell or say to the adjectives list of that character

            if token.dep_ == "ROOT" and token.lemma_ in ["do", "say", "act", "think", "feel"]:
                # Extract actions, e.g., "John ran", "Sarah shouted"
                for char in characters_in_sentence:
                    character_info[char]["actions"].append(sent.text)




print(character_info['Pablo']["description"])
print(character_info['Juan']["description"])

['alto', 'guapo']
['bajo', 'feo']


In [14]:
agata_text = "-¿Qué sintió al saber lo que había hecho?-preguntó Armstrong-.¿No lo lamentó?¿No se reprochó su conducta? \
    Lombrad exclamó:-¿Yo? No tenía nada que reprocharme."

nlp = spacy.load('es_core_news_md')

# Preprocess the text
doc = nlp(agata_text)

# Initialize containers to store character information
character_info = defaultdict(lambda: {
    "description": [],
    "sentences": [],
    "mentions": 0,
    "relationships": defaultdict(int),
    "actions": []
})

# Iterate through the sentences to extract relevant information
for sent in doc.sents:
    print("----")
    print(sent)
    sent_doc = nlp(sent.text)
    characters_in_sentence = []
    
    # We analyze the sentence token by token
    for token in sent_doc:
        # if there is an entity of type person we add all adjectives in the sentence as the description of the entity, until we find another person
        if token.ent_type_ == "PER":
            character_name = token.text
            
            # Add the sentence to the sentences list of that character
            character_info[character_name]["sentences"] += sent.text

            # Add the adjectives to the adjectives list of that character
            character_info[character_name]["description"] += search_adj_for_person(character_name,sent_doc)



print(character_info)

----
-¿Qué sintió al saber lo que había hecho?-preguntó Armstrong-.¿No lo lamentó?¿No se reprochó su conducta?     
----
Lombrad exclamó:-¿Yo?
----
No tenía nada que reprocharme.
defaultdict(<function <lambda> at 0x0000027CEBB914E0>, {'Lombrad': {'description': [], 'sentences': ['L', 'o', 'm', 'b', 'r', 'a', 'd', ' ', 'e', 'x', 'c', 'l', 'a', 'm', 'ó', ':', '-', '¿', 'Y', 'o', '?'], 'mentions': 0, 'relationships': defaultdict(<class 'int'>, {}), 'actions': []}})


In [25]:
spacy.explain("ROOT")

'root'

In [ ]:
# Resumir info
from transformers import pipeline

# Cargar un pipeline de resumen
resumir = pipeline('summarization')

# Crear resumenes cortos de las oraciones donde aparece cada personaje
for personaje, oraciones in info_personajes.items():
    print(f"Personaje: {personaje}")
    texto_unido = " ".join(oraciones)
    resumen = resumir(texto_unido[:1000], max_length=100, min_length=30, do_sample=False)
    print(f"Resumen: {resumen[0]['summary_text']}")